## Multi-fidelity Bayesian Optimization

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math

import pandas as pd
import torch

from botorch.test_functions.multi_fidelity import AugmentedHartmann
problem = AugmentedHartmann()

def test_function(input_dict):
    x = torch.tensor([input_dict[f"x{i}"] for i in range(1,7)])
    s = torch.tensor(input_dict["s"]).unsqueeze(0)
    return {"f": problem(torch.cat((x,s))).numpy()}


# define vocs
from xopt import VOCS
vocs = VOCS(
    variables={
        "x1": [0, 1],
        "x2": [0, 1],
        "x3": [0, 1],
        "x4": [0, 1],
        "x5": [0, 1],
        "x6": [0, 1],
    },
    objectives={"f": "MINIMIZE"},
)

In [2]:
# create xopt object
from xopt.generators import MultiFidelityBayesianGenerator
from xopt import Evaluator, Xopt

# get and modify default generator options
options = MultiFidelityBayesianGenerator.default_options()

# set the base cost of evaluating the function
options.acq.base_cost = 5.0
options.acq.fidelities = [0.5,0.75,1.0]

generator = MultiFidelityBayesianGenerator(vocs, options=options)
evaluator = Evaluator(function=test_function)

X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator)
X.options.strict = True

In [3]:
# evaluate random initial points at mixed fidelities to seed optimization and view data
X.evaluate_data(
    pd.DataFrame(np.random.rand(5,7), columns=X.vocs.variable_names + ["s"])
)
X.data

,x1,x2,x3,x4,x5,x6,s,f,xopt_runtime,xopt_error
1,0.779599,0.510060,0.149707,0.293865,0.884184,0.675545,0.810736,-0.0029628815,0.001316,False
2,0.447131,0.720012,0.353257,0.660626,0.750080,0.451302,0.921690,-0.22127305,0.000212,False
3,0.127085,0.415934,0.858427,0.462430,0.012538,0.889702,0.703613,-1.1211394,0.000175,False
4,0.643801,0.781307,0.506795,0.742759,0.389843,0.615218,0.462640,-0.059958965,0.000176,False
5,0.900511,0.655916,0.355355,0.106642,0.944645,0.981492,0.042277,-0.0003087785,0.000174,False


In [4]:
# get the total cost of previous observations (base_cost + s for each measurement)
X.generator.calculate_total_cost()

27.940956222381303

In [5]:
# run optimization until the cost budget is exhausted
budget = 100
while X.generator.calculate_total_cost() < budget:
    print("step")
    X.step()

step
step
step
step
step
step
step
step
step
step
step
step
step
step


In [6]:
X.data

,x1,x2,x3,x4,x5,x6,s,f,xopt_runtime,xopt_error
1,0.779599,0.510060,0.149707,0.293865,0.884184,0.675545,0.810736,-0.0029628815,0.001316,False
2,0.447131,0.720012,0.353257,0.660626,0.750080,0.451302,0.921690,-0.22127305,0.000212,False
3,0.127085,0.415934,0.858427,0.462430,0.012538,0.889702,0.703613,-1.1211394,0.000175,False
4,0.643801,0.781307,0.506795,0.742759,0.389843,0.615218,0.462640,-0.059958965,0.000176,False
5,0.900511,0.655916,0.355355,0.106642,0.944645,0.981492,0.042277,-0.0003087785,0.000174,False
6,0.864225,0.568073,0.183885,0.515662,0.667358,0.644038,0.500000,-0.029905917,0.000232,False
7,0.778452,0.420054,0.090288,0.336781,0.903650,0.435032,0.500000,-0.0044433833,0.000289,False
8,0.788300,0.669227,0.220204,0.190993,0.784073,0.514197,0.500000,-0.0091581,0.000315,False
9,0.660233,0.560095,0.025465,0.268609,0.747344,0.690535,0.500000,-0.028843867,0.000221,False
10,0.806009,0.687624,0.067081,0.379974,0.970975,0.636780,0.500000,-0.00083629217,0.000312,False


In [7]:
X.generator.calculate_total_cost()

105.6909562223813